# GQN With DGF - Pool Representation and 8 Generative Steps

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from gqn import GenerativeQueryNetwork, partition
from dataset import GQN_Dataset
from torch.distributions import Normal
import shutil
import os
import numpy as np
import cv2
import torch
import glob

In [ ]:
df = pd.read_csv("main.log")
print("Total steps: {}".format(df["step"].tolist()[-1]))
models = glob.glob("checkpoints/checkpoint_model_*.pth")
models.sort(key=lambda x: os.path.getmtime(x))
print("Last model checkpoint: {}".format(models[-1]))

In [ ]:
df = df[df["step"]>100]
step = df["step"].to_list()
ll = df["ll"].to_list()
kl = df["kl"].to_list()
elbo = df['elbo'].to_list()
sigma = df['sigma'].to_list()
mu = df['mu'].to_list()
kl_av = []
ll_av = []
elbo_av = []

average_step = 100
for i in range(average_step, len(step)):
    kl_av.append(sum(kl[i-average_step:i])/average_step)
    ll_av.append(sum(ll[i-average_step:i])/average_step)
    elbo_av.append(sum(elbo[i-average_step:i])/average_step)
step = step[average_step:]
sigma = sigma[average_step:]
mu =mu[average_step:]

plt.figure(num=None, figsize=(18, 12))
plt.subplot(231)
plt.plot(step, elbo_av)
plt.subplot(232)
plt.plot(step, ll_av)
plt.subplot(233)
plt.plot(step, kl_av)

plt.subplot(234)
plt.plot(step, mu)
plt.subplot(235)
plt.plot(step, sigma)
plt.show()

In [ ]:
valid_dataset = GQN_Dataset(root_dir="data/rooms_ring_camera/", train=False)
x, v = valid_dataset[1]
x = x.view((1, *x.shape))
v = v.view((1, *v.shape))

max_m=5
x, v, x_q, v_q = partition(x, v, max_m, 5)
batch, *_ = x.shape
print(x.shape)
print(v.shape)
print(x_q.shape)
device = torch.device("cpu")
model = GenerativeQueryNetwork(x_dim=3, v_dim=7, r_dim=256, h_dim=128, dgf_dim=256, z_dim=64, L=8, pool=True).to(device)
checkpoint = torch.load(models[-1])
# checkpoint = torch.load("./checkpoints/checkpoint_model_50000.pth") #
# checkpoint = torch.load("./chkpnts/checkpoint_model_500000.pth") #
model.load_state_dict(checkpoint)

x_mu = model.sample(x, v, v_q)
# x_mu, r, kl = model(x, v, x_q, v_q)
x_mu = x_mu.detach().cpu().numpy()
x = x.numpy()
x_q = x_q.numpy()

x = np.moveaxis(x, [2, 3, 4], [-1, 2, 3])
x_q = np.moveaxis(x_q, [1, 2, 3], [-1, -3, -2])
x_mu = np.moveaxis(x_mu, [1, 2, 3], [-1, -3, -2])

n = 30
for i in range(n):
    full = np.concatenate(x[i], axis=1)
    space = np.ones(shape=(full.shape[0], 10, 3))
    full = np.concatenate([full, space, x_mu[i], space, x_q[i]], axis=1)
    plt.figure(num=None, figsize=(18, 6))
    plt.imshow(full)
    plt.show()



In [ ]:
device = torch.device("cpu")
model1 = GenerativeQueryNetwork(x_dim=3, v_dim=7, r_dim=256, h_dim=128, dgf_dim=256, z_dim=64, L=8, pool=True).to(device)
model2 = GenerativeQueryNetwork(x_dim=3, v_dim=7, r_dim=256, h_dim=128, dgf_dim=256, z_dim=64, L=8, pool=True).to(device)

# checkpoint = torch.load(models[-1])
checkpoint1 = torch.load("checkpoints/checkpoint_model_30000.pth") #
checkpoint2 = torch.load("checkpoints/checkpoint_model_10000.pth") #
model1.load_state_dict(checkpoint1)
model2.load_state_dict(checkpoint2)

i1 = model1.input_layer.weight.data
i2 = model1.input_layer.weight.data
print(torch.sum(i1))
print(torch.sum(i2))
print(i1-i2)

o1 = model1.output_layer.weight.data
o2 = model1.output_layer.weight.data
print(torch.sum(o1))
print(torch.sum(o2))
print(o1-o2)
